In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
## get crypto data

import requests
import pandas as pd

ticker_price = requests.get('https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT')
print(ticker_price.json())



{'symbol': 'BTCUSDT', 'price': '84888.31000000'}


In [3]:
tickers_prices = requests.get('https://api.binance.com/api/v3/ticker/price')

pd.DataFrame(tickers_prices.json()).head(20)

,symbol,price
0,ETHBTC,0.01869000
1,LTCBTC,0.00088500
2,BNBBTC,0.00696900
3,NEOBTC,0.00006440
4,QTUMETH,0.00124600
5,EOSETH,0.00039460
6,SNTETH,0.00001700
7,BNTETH,0.00020260
8,BCCBTC,0.00000000
9,GASBTC,0.00004020


In [10]:
def get_klines(symbol: str, interval:str="1d", limit:int=7):
    klines = requests.get(f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}')

    klines_df = pd.DataFrame(klines.json())
    
    klines_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    klines_df['open_time'] = pd.to_datetime(klines_df['open_time'], unit='ms')
    klines_df['close_time'] = pd.to_datetime(klines_df['close_time'], unit='ms')
    klines_df = klines_df[['open_time', 'close_time', 'volume', 'low', 'close']]
    
    return klines_df

klines_df = get_klines('BTCUSDT')
klines_df

,open_time,close_time,volume,low,close
0,2025-04-11,2025-04-11 23:59:59.999,34435.43797000,78969.58000000,83423.84000000
1,2025-04-12,2025-04-12 23:59:59.999,18470.74437000,82792.95000000,85276.90000000
2,2025-04-13,2025-04-13 23:59:59.999,24680.04181000,83034.23000000,83760.00000000
3,2025-04-14,2025-04-14 23:59:59.999,28659.09348000,83678.00000000,84591.58000000
4,2025-04-15,2025-04-15 23:59:59.999,20910.99528000,83600.00000000,83643.99000000
5,2025-04-16,2025-04-16 23:59:59.999,20867.24519000,83111.64000000,84030.38000000
6,2025-04-17,2025-04-17 23:59:59.999,13413.07521000,83736.26000000,84916.06000000


In [61]:
## get news data 
import requests

# === Config ===
API_KEY = ''  # Replace with your API key
PAIR = 'BTCUSDT'  # You can change to ETHUSDT, XRPUSDT, etc.

# You can map pairs to search keywords if needed
pair_keywords = {
    'BTCUSDT': 'bitcoin',
    'ETHUSDT': 'ethereum',
    'XRPUSDT': 'ripple',
    # Add more if needed
}

def get_crypto_news_from_cryptopanic(keyword):
    url = 'https://cryptopanic.com/api/v1/posts/'
    params = {
        'auth_token': API_KEY,
        'currencies': keyword,
        'kind': 'news',  # other options: 'media', 'all'
        'filter': 'important',  # other options: 'rising', 'bullish', 'bearish', etc.
    }

    data = requests.get(url, params=params).json()
    result = data['results']
    for i in range(data['count']):
        url = data['next']

        if url is None:
            break
            
        data = requests.get(url).json()
        result.extend(data['results'])

    return result


news = get_crypto_news_from_cryptopanic('bitcoin')
news

[{'kind': 'news',
  'domain': 'cryptoslate.com',
  'votes': {'negative': 0,
   'positive': 5,
   'important': 3,
   'liked': 5,
   'disliked': 0,
   'lol': 0,
   'toxic': 0,
   'saved': 0,
   'comments': 1},
  'source': {'title': 'CryptoSlate',
   'region': 'en',
   'domain': 'cryptoslate.com',
   'path': None,
   'type': 'feed'},
  'title': 'Family offices show stronger preference for Ethereum ETFs over Bitcoin',
  'published_at': '2025-04-16T21:30:10Z',
  'slug': 'Family-offices-show-stronger-preference-for-Ethereum-ETFs-over-Bitcoin',
  'currencies': [{'code': 'BTC',
    'title': 'Bitcoin',
    'slug': 'bitcoin',
    'url': 'https://cryptopanic.com/news/bitcoin/'},
   {'code': 'ETH',
    'title': 'Ethereum',
    'slug': 'ethereum',
    'url': 'https://cryptopanic.com/news/ethereum/'}],
  'id': 21115646,
  'url': 'https://cryptopanic.com/news/21115646/Family-offices-show-stronger-preference-for-Ethereum-ETFs-over-Bitcoin?mtm_campaign=API-OFA',
  'created_at': '2025-04-16T21:30:10Z'},

In [62]:
len(news)

200

In [72]:
from datetime import datetime, timedelta

datetime.today() - timedelta(days=7)

datetime.datetime(2025, 4, 11, 15, 57, 41, 319933)

In [80]:
pd.Timedelta

pandas._libs.tslibs.timedeltas.Timedelta

In [113]:
limit = 7

news_df = pd.DataFrame(news)
news_df['published_at'] = news_df['published_at'].apply(lambda x: pd.to_datetime(x).to_pydatetime())
news_df['positive'] = news_df['votes'].apply(lambda x: x['positive'])
news_df['important'] = news_df['votes'].apply(lambda x: x['important'])
news_df['liked'] = news_df['votes'].apply(lambda x: x['liked'])

news_df = news_df[['published_at', 'title', 'positive', 'important', 'liked']]
news_df.sort_values(['published_at', 'important'], ascending=False)

news_df

,published_at,title,positive,important,liked
0,2025-04-16 21:30:10+00:00,Family offices show stronger preference for Et...,5,3,5
1,2025-04-16 15:44:47+00:00,Ethereum Is What Bitcoin Was Meant to Be,7,4,7
2,2025-04-14 14:30:40+00:00,"MicroStrategy Acquires 3,459 Bitcoins for $285...",8,5,6
3,2025-04-13 14:44:50+00:00,Saylor signals new Bitcoin buy after Strategy ...,6,4,5
4,2025-04-11 15:18:14+00:00,Sweden: proposal to include Bitcoin in nationa...,16,5,8
...,...,...,...,...,...
195,2024-12-09 15:26:02+00:00,El Salvador Bitcoin Policy Faces Shift for $1....,0,3,1
196,2024-12-09 08:59:06+00:00,Bitcoin and Ethereum ETFs Break Records Amid P...,14,11,13
197,2024-12-09 08:47:05+00:00,Amazon Shareholders Push for Bitcoin to Fight ...,14,10,13
198,2024-12-08 21:25:50+00:00,Amazon shareholders tells company to consider ...,29,15,22


In [107]:
news_df['published_at'] > curr_date

TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and datetime

In [112]:
news_df['published_at']

0     2025-04-16 21:30:10+00:00
1     2025-04-16 15:44:47+00:00
2     2025-04-14 14:30:40+00:00
3     2025-04-13 14:44:50+00:00
4     2025-04-11 15:18:14+00:00
                 ...           
195   2024-12-09 15:26:02+00:00
196   2024-12-09 08:59:06+00:00
197   2024-12-09 08:47:05+00:00
198   2024-12-08 21:25:50+00:00
199   2024-12-08 19:18:47+00:00
Name: published_at, Length: 200, dtype: datetime64[ns, UTC]

In [ ]:
## get twitter data

In [17]:
import requests

# === Config ===
API_KEY = 'your_lunarcrush_api_key_here'
SYMBOL = 'BTC'

def get_lunarcrush_social_data(symbol='BTC'):
    url = 'https://api.lunarcrush.com/v2'
    params = {
        'data': 'assets',
        'key': API_KEY,
        'symbol': symbol,
    }

    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code != 200 or 'data' not in data:
        print("Error fetching data:", data)
        return {}

    asset = data['data'][0]
    return {
        'name': asset.get('name'),
        'symbol': asset.get('symbol'),
        'galaxy_score': asset.get('galaxy_score'),
        'alt_rank': asset.get('alt_rank'),
        'twitter_volume': asset.get('twitter_volume'),
        'news_score': asset.get('news_score'),
        'social_score': asset.get('social_score'),
        'url_shares': asset.get('url_shares')
    }

# === Example Usage ===
if __name__ == "__main__":
    social_data = get_lunarcrush_social_data('BTC')
    print("🚀 Social Insights for BTC:")
    for key, value in social_data.items():
        print(f" - {key.replace('_', ' ').title()}: {value}")

ConnectionError: HTTPSConnectionPool(host='api.lunarcrush.com', port=443): Max retries exceeded with url: /v2?data=assets&key=your_lunarcrush_api_key_here&symbol=BTC (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1337d6c90>: Failed to resolve 'api.lunarcrush.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
# lunarcrush не подходит

In [22]:
import requests

url = "https://lunarcrush.com/api4/public/topic/bitcoin/posts/v1?start=1744138800&end=1744743600"
headers = {
  'Authorization': 'Bearer '
}

response = requests.request("GET", url, headers=headers)

print(response.text.encode('utf8'))

b'{"error":"You must have an active Discover or higher subscription to use this endpoint."}'


In [45]:
r = requests.get('https://feed.defillama.com')
r

<Response [403]>

In [116]:
from py_clob_client.client import ClobClient

In [ ]:
client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=POLYMARKET_PROXY_ADDRESS)

In [ ]:
# Доделать выбор даты start_date, end_date!

In [158]:
DT_FORMAT = '%Y-%m-%d'

def add_days(dt, days, dt_format=DT_FORMAT):
    return datetime.strftime(datetime.strptime(dt, dt_format) + timedelta(days=days), dt_format)


add_days('2025-04-17', -7)

'2025-04-10'

In [212]:
def get_klines(symbol: str, start_date: str, end_date: str, interval: str = "1d") -> pd.DataFrame:
    sdt = int(datetime.strptime(start_date, DT_FORMAT).timestamp() * 1000)
    edt = int(datetime.strptime(end_date, DT_FORMAT).timestamp() * 1000)

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": sdt,
        "endTime": edt
    }
    
    klines = requests.get('https://api.binance.com/api/v3/klines', params=params)
    klines_df = pd.DataFrame(klines.json())

    klines_df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                         'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    klines_df['open_time'] = pd.to_datetime(klines_df['open_time'], unit='ms')
    klines_df['close_time'] = pd.to_datetime(klines_df['close_time'], unit='ms')
    for col in ['open', 'high', 'low', 'close', 'volume']:
        klines_df[col] = klines_df[col].apply(lambda x: float(x))

    klines_df['date'] = klines_df['open_time']
    klines_df = klines_df[['date', 'close_time', 'volume', 'open', 'close']]
    klines_df['diff'] = klines_df['close'] - klines_df['open']

    return klines_df


end_date = '2025-04-17'
start_date = add_days(end_date, -7)
klines_df = get_klines('BTCUSDT', start_date, end_date)
klines_df.sort_values("date")

,date,close_time,volume,open,close,diff
0,2025-04-10,2025-04-10 23:59:59.999,33284.80718,82615.22,79607.30,-3007.92
1,2025-04-11,2025-04-11 23:59:59.999,34435.43797,79607.30,83423.84,3816.54
2,2025-04-12,2025-04-12 23:59:59.999,18470.74437,83423.83,85276.90,1853.07
3,2025-04-13,2025-04-13 23:59:59.999,24680.04181,85276.91,83760.00,-1516.91
4,2025-04-14,2025-04-14 23:59:59.999,28659.09348,83760.00,84591.58,831.58
5,2025-04-15,2025-04-15 23:59:59.999,20910.99528,84591.58,83643.99,-947.59
6,2025-04-16,2025-04-16 23:59:59.999,20867.24519,83643.99,84030.38,386.39


In [224]:
def get_crypto_news_from_cryptopanic(
        keyword: str,
        start_date: str,
        end_date: str,
        kind: str = 'news',
        filter: str = 'important',
) -> pd.DataFrame:
    url = 'https://cryptopanic.com/api/v1/posts/'
    params = {
        'auth_token': os.environ["CRYPTOPANIC_API_KEY"],
        'currencies': keyword,
        'kind': kind,
        'filter': filter
    }

    data = requests.get(url, params=params).json()
    result = data['results']
    
    for i in range(data['count']):
        url = data['next']

        if url is None:
            break

        data = requests.get(url).json()
        result.extend(data['results'])

    news_df = pd.DataFrame(result)
    for col in ['positive', 'important', 'liked']:
        news_df[col] = news_df['votes'].apply(lambda x: x[col])

    news_df = news_df[['published_at', 'title', 'positive', 'important', 'liked']]
    news_df['published_at'] = pd.to_datetime(news_df['published_at']).dt.tz_convert(None)
    
    news_df['date'] = news_df['published_at'].apply(lambda x: str(x)[:10])
    news_df = news_df.drop(columns=['published_at'])
    news_df = news_df[(start_date <= news_df['date']) & (news_df['date'] <= end_date)]

    return news_df.sort_values(['date', 'important'], ascending=False)


end_date = '2025-04-17'
start_date = add_days(end_date, -7)
news_df = get_crypto_news_from_cryptopanic('BTC', start_date, end_date)
news_df

,title,positive,important,liked,date
1,Ethereum Is What Bitcoin Was Meant to Be,8,4,8,2025-04-16
0,Family offices show stronger preference for Et...,5,3,6,2025-04-16
2,"MicroStrategy Acquires 3,459 Bitcoins for $285...",8,5,6,2025-04-14
3,Saylor signals new Bitcoin buy after Strategy ...,6,4,5,2025-04-13
4,Sweden: proposal to include Bitcoin in nationa...,16,5,8,2025-04-11
5,"Crypto Market Rises Sharply, but Experts Warn ...",0,4,0,2025-04-10
6,Charles Hoskinson sees Bitcoin hitting $250K a...,5,3,4,2025-04-10


In [ ]:
# теперь надо закодить ai agent
